# 06 - Statistiche Wikipedia

### Generazione Links Pagine Wikipedia dei giocatori

In [13]:
import re
import numpy as np
import pandas as pd
from requests import get
from pandas import read_csv
from unidecode import unidecode
from bs4 import BeautifulSoup as BS

In [3]:
# dfNames = read_csv('tables/nome_nomechiave.csv')
# dfNames.head()

,nome,nome_chiave
0,Kylian Mbappé,Mbappé
1,Neymar,Neymar
2,Leo Messi,Messi
3,Mohamed Salah,Salah
4,Harry Kane,Harry Kane


In [4]:
# s1 = 'https://www.google.com/search?&q='
# s2 = '+site%3Aen.wikipedia.org'
    
# dfNames['urls'] = [s1+x.replace("'", '').replace('-',' ').replace(' ','+')+s2 for x in dfNames.nome]
# dfNames.head()

,nome,nome_chiave,urls
0,Kylian Mbappé,Mbappé,https://www.google.com/search?&q=Kylian+Mbappé...
1,Neymar,Neymar,https://www.google.com/search?&q=Neymar+site%3...
2,Leo Messi,Messi,https://www.google.com/search?&q=Leo+Messi+sit...
3,Mohamed Salah,Salah,https://www.google.com/search?&q=Mohamed+Salah...
4,Harry Kane,Harry Kane,https://www.google.com/search?&q=Harry+Kane+si...


In [46]:
# dfNames['wiki'] = np.nan

In [66]:
# i = 0

# for url in dfNames.urls:
#     if isinstance(dfNames.wiki[i], float):
#         dfNames.iloc[i,-1] = BS(get(url).text, 'lxml').find('cite').text
#         i += 1
#     else:
#         i += 1

In [67]:
# sum(dfNames.wiki.isnull())

0

In [68]:
# dfNames.head()

,nome,nome_chiave,urls,wiki
0,Kylian Mbappé,Mbappé,https://www.google.com/search?&q=Kylian+Mbappé...,https://en.wikipedia.org/wiki/Kylian_Mbappé
1,Neymar,Neymar,https://www.google.com/search?&q=Neymar+site%3...,https://en.wikipedia.org/wiki/Neymar
2,Leo Messi,Messi,https://www.google.com/search?&q=Leo+Messi+sit...,https://en.wikipedia.org/wiki/Lionel_Messi
3,Mohamed Salah,Salah,https://www.google.com/search?&q=Mohamed+Salah...,https://en.wikipedia.org/wiki/Mohamed_Salah
4,Harry Kane,Harry Kane,https://www.google.com/search?&q=Harry+Kane+si...,https://en.wikipedia.org/wiki/Harry_Kane


In [70]:
# dfNames.to_csv('tables/wiki_urls.csv',index=False)

### Scraping Wikipedia Stats

In [77]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep

In [84]:
df = read_csv('tables/Top300_Urls - Sheet1.csv')
df = df[['nome','wiki']]
df.head()

,nome,wiki
0,Kylian Mbappé,https://en.wikipedia.org/wiki/Kylian_Mbappé
1,Neymar,https://en.wikipedia.org/wiki/Neymar
2,Lionel Messi,https://en.wikipedia.org/wiki/Lionel_Messi
3,Mohamed Salah,https://en.wikipedia.org/wiki/Mohamed_Salah
4,Harry Kane,https://en.wikipedia.org/wiki/Harry_Kane


In [85]:
s1 = 'https://tools.wmflabs.org/pageviews/?project=en.wikipedia.org&platform=all-access&agent=user&start=2018-01-01&end=2018-12-20&pages='
df['urls'] = [s1 + x.split('/')[-1] for x in df.wiki]
df['wiki_visual'] = np.nan
df['wiki_avg'] = np.nan
df.head()

,nome,wiki,urls,wiki_visual,wiki_avg
0,Kylian Mbappé,https://en.wikipedia.org/wiki/Kylian_Mbappé,https://tools.wmflabs.org/pageviews/?project=e...,NaN,NaN
1,Neymar,https://en.wikipedia.org/wiki/Neymar,https://tools.wmflabs.org/pageviews/?project=e...,NaN,NaN
2,Lionel Messi,https://en.wikipedia.org/wiki/Lionel_Messi,https://tools.wmflabs.org/pageviews/?project=e...,NaN,NaN
3,Mohamed Salah,https://en.wikipedia.org/wiki/Mohamed_Salah,https://tools.wmflabs.org/pageviews/?project=e...,NaN,NaN
4,Harry Kane,https://en.wikipedia.org/wiki/Harry_Kane,https://tools.wmflabs.org/pageviews/?project=e...,NaN,NaN


In [91]:
driver = webdriver.Chrome(executable_path='chromedriver')
driver.maximize_window()

#=======================

i = 0
for url in df.urls:
    if isinstance(df.wiki_visual[i], float):
        driver.get(url)
        sleep(3)       
        try:
            stats = driver.find_elements_by_class_name('linear-legend--counts')
            d = dict([tuple(x.text.split(':')) for x in stats if x.text])
            df.loc[i, 'wiki_visual'] = d['Visualizzazioni per pagina'].strip().replace(".","")
            df.loc[i, 'wiki_avg'] = d['Media giornaliera'].strip().replace(".","")
            print(i, end=' ')
            i += 1
        except:
            i += 1
    else:
        i += 1

#=======================

driver.quit()

114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 241 242 243 245 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 

In [92]:
df.head(10)

,nome,wiki,urls,wiki_visual,wiki_avg
0,Kylian Mbappé,https://en.wikipedia.org/wiki/Kylian_Mbappé,https://tools.wmflabs.org/pageviews/?project=e...,6854994,19364
1,Neymar,https://en.wikipedia.org/wiki/Neymar,https://tools.wmflabs.org/pageviews/?project=e...,6094579,17216
2,Lionel Messi,https://en.wikipedia.org/wiki/Lionel_Messi,https://tools.wmflabs.org/pageviews/?project=e...,11563642,32666
3,Mohamed Salah,https://en.wikipedia.org/wiki/Mohamed_Salah,https://tools.wmflabs.org/pageviews/?project=e...,6538061,18469
4,Harry Kane,https://en.wikipedia.org/wiki/Harry_Kane,https://tools.wmflabs.org/pageviews/?project=e...,5013018,14161
5,Antoine Griezmann,https://en.wikipedia.org/wiki/Antoine_Griezmann,https://tools.wmflabs.org/pageviews/?project=e...,3221342,9100
6,Kevin De Bruyne,https://en.wikipedia.org/wiki/Kevin_De_Bruyne,https://tools.wmflabs.org/pageviews/?project=e...,1644090,4644
7,Philippe Coutinho,https://en.wikipedia.org/wiki/Philippe_Coutinho,https://tools.wmflabs.org/pageviews/?project=e...,2619788,7401
8,Eden Hazard,https://en.wikipedia.org/wiki/Eden_Hazard,https://tools.wmflabs.org/pageviews/?project=e...,3049334,8614
9,Paulo Dybala,https://en.wikipedia.org/wiki/Paulo_Dybala,https://tools.wmflabs.org/pageviews/?project=e...,1678324,4741


In [94]:
df.to_csv('columns/wiki_visual_and_avg.csv',index=False,encoding='ansi')